In [1]:
import cv2
import numpy as np
import pandas as pd
import os

from sklearn.cluster import KMeans

from imutils import face_utils
import dlib

In [2]:
# os.environ["OMP_NUM_THREADS"] = '1'
import warnings
warnings.filterwarnings('ignore')

In [3]:
n_colors = 4

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
data_root = './images/dataset__'
seasons = ['spring', 'summer', 'fall', 'winter']

In [5]:
def bgr2cmyk(bgr):
    res_cmyk = []
    bgr = bgr / 255.0
    
    for i in range(len(bgr)):
        b, g, r = bgr[i]
        k = 1 - max(r, g, b)
        c = (1 - r - k) / (1 - k) * 255
        m = (1 - g - k) / (1 - k) * 255
        y = (1 - b - k) / (1 - k) * 255

        tmp = np.array([c, m, y, k])
        res_cmyk.append(tmp)
        
    return np.array(res_cmyk)

In [6]:
# # lips = None
# # left_cheek = None
# # right_cheek = None
# # right_eye = None
# # left_eye = None
# # nose = None

# def extract_face_part(img, face_part_points):

#     (x, y, w, h) = cv2.boundingRect(face_part_points)
#     crop = img[y:y+h, x:x+w]
    
#     # https://www.researchgate.net/publication/262371199_Explicit_image_detection_using_YCbCr_space_color_model_as_skin_detection
#     # filter skin only (YCbCr)
#     crop = cv2.cvtColor(crop, cv2.COLOR_BGR2YCrCb)
#     mask = cv2.inRange(crop, np.array([0, 133, 77]), np.array([255, 173, 127]))
#     crop = cv2.bitwise_and(crop, crop, mask=mask)
#     crop = cv2.cvtColor(crop, cv2.COLOR_YCrCb2BGR)

#     crop = crop[~np.all(crop == [0, 135, 0], axis=-1)]
#     crop = crop.reshape(((1, crop.shape[0], 3)))
    
#     return crop

# def detect_face_part(img):
#     face_parts = [[] for _ in range(len(face_utils.FACIAL_LANDMARKS_IDXS))]

#     faces = detector(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 1)
    
#     if len(faces) == 0:
#         return 0

#     rect = faces[0]

#     shape = predictor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), rect)
#     shape = face_utils.shape_to_np(shape)

#     for idx, (_, (i, j)) in enumerate(face_utils.FACIAL_LANDMARKS_IDXS.items()):
#         if idx not in [1, 3]:
#             face_parts[idx] = shape[i:j]

#     lips = extract_face_part(img, np.concatenate((shape[48:60], shape[60:68])))
#     left_cheek = extract_face_part(img, np.concatenate((shape[29:33], shape[4:9])))
#     right_cheek = extract_face_part(img, np.concatenate((shape[29:33], shape[10:15])))
#     right_eye = extract_face_part(img, shape[36:42])
#     left_eye = extract_face_part(img, shape[42:48])
#     nose = extract_face_part(img, shape[27:36])

#     return len(faces), lips, left_cheek, right_cheek, right_eye, left_eye, nose

# def create_palette(img, image_path='image.jpg'):
        
#     img = cv2.imread(image_path)
    
#     if img is None:
#         os.remove(image_path)
#         return None
        
#     yes_faces, lips, left_cheek, right_cheek, right_eye, left_eye, nose = detect_face_part(img)
    
#     if not yes_faces:
#         os.remove(image_path)
#         return None
    
#     stacked_images = np.hstack([right_eye, left_eye, lips, left_cheek, right_cheek, nose])
#     stacked_images = stacked_images.reshape(-1, 3)
    
#     if stacked_images.shape[0] == 0:
#         os.remove(image_path)
#         return None
        
#     kmeans = KMeans(n_clusters=n_colors, n_init=10, random_state=42)
#     kmeans.fit(stacked_images)

#     cluster_centers = kmeans.cluster_centers_.astype(int)
    
#     # save_palette(cluster_centers)
    
#     return cluster_centers, lips, left_cheek, right_cheek, right_eye, left_eye, nose

In [7]:
class PaletteCreator:
    def __init__(self, n_colors=3):
        
        self.n_colors = n_colors

        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        self.right_eye = None
        self.left_eye = None
        self.left_cheek = None
        self.right_cheek = None
        self.lips = None
        self.nose = None
        
    def extract_face_part(self, face_part_points):

        (x, y, w, h) = cv2.boundingRect(face_part_points)
        crop = self.img[y:y+h, x:x+w]
        
        # https://www.researchgate.net/publication/262371199_Explicit_image_detection_using_YCbCr_space_color_model_as_skin_detection
        # filter skin only (YCbCr)
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2YCrCb)
        mask = cv2.inRange(crop, np.array([0, 133, 77]), np.array([255, 173, 127]))
        crop = cv2.bitwise_and(crop, crop, mask=mask)
        crop = cv2.cvtColor(crop, cv2.COLOR_YCrCb2BGR)

        crop = crop[~np.all(crop == [0, 135, 0], axis=-1)]
        crop = crop.reshape(((1, crop.shape[0], 3)))
        
        return crop


    def detect_face_part(self):
        face_parts = [[] for _ in range(len(face_utils.FACIAL_LANDMARKS_IDXS))]

        faces = self.detector(cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY), 1)
        
        if len(faces) == 0:
            return 0

        rect = faces[0]

        shape = self.predictor(cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY), rect)
        shape = face_utils.shape_to_np(shape)

        for idx, (_, (i, j)) in enumerate(face_utils.FACIAL_LANDMARKS_IDXS.items()):
            if idx not in [1, 3]:
                face_parts[idx] = shape[i:j]

        self.lips = self.extract_face_part(np.concatenate((shape[48:60], shape[60:68])))
        self.left_cheek = self.extract_face_part(np.concatenate((shape[29:33], shape[4:9])))
        self.right_cheek = self.extract_face_part(np.concatenate((shape[29:33], shape[10:15])))
        self.right_eye = self.extract_face_part(shape[36:42])
        self.left_eye = self.extract_face_part(shape[42:48])
        self.nose = self.extract_face_part(shape[27:36])

        return len(faces)
        
    def create_palette(self, image_path='image.jpg'):
        
        self.img = cv2.imread(image_path)
        
        if self.img is None:
            os.remove(image_path)
            return None
            
        yes_faces = self.detect_face_part()
        
        if not yes_faces:
            os.remove(image_path)
            return None
        
        stacked_images = np.hstack([self.right_eye, self.left_eye, self.lips, self.left_cheek, self.right_cheek, self.nose])
        stacked_images = stacked_images.reshape(-1, 3)
        
        if stacked_images.shape[0] == 0:
            os.remove(image_path)
            return None
            
        kmeans = KMeans(n_clusters=self.n_colors, n_init=10, random_state=42)
        kmeans.fit(stacked_images)

        cluster_centers = kmeans.cluster_centers_
        
        return cluster_centers, self.lips, self.left_cheek, self.right_cheek, self.right_eye, self.left_eye, self.nose

In [8]:
pc = PaletteCreator(n_colors=4)

In [9]:
def calculate_contrast(img):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    hist /= hist.sum()
    mean = np.mean(hist)
    variance = np.mean((hist - mean) ** 2)
    return variance

In [10]:
for season in seasons:
    this_df = pd.DataFrame(columns=['lips_s', 'face_l_var', 'skin_avg_b', 'R', 'G', 'B', 'H', 'S', 'V', 'L', 'a', 'b', 'label'])
    for file in os.listdir(os.path.join(data_root, season)):
        full_path = os.path.join(data_root, season, file)
        img = cv2.imread(full_path)
        if img is None:
            print(f'Error reading {full_path}')
            continue
        try:
            palette, lips, left_cheek, right_cheek, right_eye, left_eye, nose = pc.create_palette(full_path)
        except:
            print(f'Error reading {full_path}')
            continue
        palette = np.array([palette], np.uint8)
        hsv_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2HSV)
        lab_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2LAB)
    
        mean_hsv = np.mean(hsv_palette, axis=1)[0]
        mean_lab = np.mean(lab_palette, axis=1)[0]
        
        skin = np.hstack([left_cheek, right_cheek])
        skin = skin.reshape(-1, 3)
        kmeans = KMeans(n_clusters=10, n_init=10, random_state=42)
        kmeans.fit(skin)
        skin_centers_ = kmeans.cluster_centers_
        skin_centers = np.array([skin_centers_], np.uint8)
        lab_palette = cv2.cvtColor(skin_centers, cv2.COLOR_BGR2LAB)
        mean_lab_skin = np.mean(lab_palette, axis=1)[0]
        skin_centers_ = np.mean(skin_centers_, axis=0)
        
        kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
        lips = lips.reshape(-1, 3)
        kmeans.fit(lips)
        lips_centers = kmeans.cluster_centers_
        lips_centers = np.array([lips_centers], np.uint8)
        lab_palette = cv2.cvtColor(lips_centers, cv2.COLOR_BGR2LAB)
        mean_lab_lips = np.mean(lab_palette, axis=1)[0]
        
        h, w = img.shape[:2]
        try:
            face = detector(img)[0]
        except:
            continue
        face = img[max(0, face.top()):min(face.bottom(), h), max(0, face.left()):min(face.right(), w)]
        face_l_var = calculate_contrast(face)
        
        tmp = np.array([mean_lab_lips[1], face_l_var, mean_lab_skin[2]])
        row = np.concatenate((tmp, skin_centers_, mean_hsv, mean_lab)).tolist()
        row.append(season)
        this_df.loc[len(this_df)] = row
    print(f'{season}: {len(this_df)} rows')
    this_df.to_csv(f'images/dataset__/mean_{season}.csv', index=False)

Error reading ./images/dataset__\spring\Screenshot 2024-04-16 063001.png
Error reading ./images/dataset__\spring\Screenshot＿20201220－021100＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021239＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021339＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021427＿Chrome.jpg
Error reading ./images/dataset__\spring\강남퍼스널컬러컬러연_봄웜톤연예인9.jpg
Error reading ./images/dataset__\spring\다운로드_(39).jpg
Error reading ./images/dataset__\spring\봄웜톤11.jpg
Error reading ./images/dataset__\spring\아이유_고채도.png
Error reading ./images/dataset__\spring\아이유_노랑.png
Error reading ./images/dataset__\spring\아이유_명도.png
Error reading ./images/dataset__\spring\아이유_저명도3.png
Error reading ./images/dataset__\spring\아이유레드.png
Error reading ./images/dataset__\spring\아이유쿨.png
Error reading ./images/dataset__\spring\첫페이지.png
spring: 43 rows
Error reading ./images/dataset__\summer\1200px-180320_이유비.jpg
Error reading .

In [11]:
df = pd.concat([pd.read_csv(f'images/dataset__/mean_{season}.csv') for season in seasons])
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('images/dataset__/mean_shuffled.csv', index=False)

In [12]:
warm_only_df = df[df['label'].isin(['spring', 'fall'])]
cool_only_df = df[df['label'].isin(['summer', 'winter'])]

warm_only_df.to_csv('images/dataset__/mean_warm.csv', index=False)
cool_only_df.to_csv('images/dataset__/mean_cool.csv', index=False)

In [13]:
warm_cool_df = pd.concat([warm_only_df, cool_only_df])
warm_cool_df = warm_cool_df.sample(frac=1).reset_index(drop=True)
warm_cool_df.to_csv('images/dataset__/mean_warm_cool.csv', index=False)

In [14]:
warm_only_df =pd.read_csv('images/dataset__/mean_warm.csv')

In [15]:
warm_only_df

,lips_s,face_l_var,skin_avg_b,R,G,B,H,S,V,L,a,b,label
0,151.666667,0.000008,145.8,112.914310,133.965925,175.651445,9.75,105.75,165.50,139.25,141.50,145.25,fall
1,150.000000,0.000025,146.3,126.187999,147.715856,191.952797,8.25,99.50,181.00,152.50,142.75,144.50,spring
2,154.000000,0.000012,141.8,112.976115,125.444865,172.908069,5.25,107.25,157.75,127.75,144.50,140.50,spring
3,152.333333,0.000009,145.1,105.951249,125.220850,170.398417,8.75,115.50,163.75,134.25,143.00,145.00,fall
4,151.666667,0.000015,145.2,108.123511,128.183753,173.363279,8.50,115.75,167.00,136.25,143.50,145.25,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,151.333333,0.000023,143.6,126.972631,145.270463,185.783577,8.75,87.25,178.50,153.25,141.00,143.00,spring
85,153.666667,0.000013,147.0,85.250280,103.801229,156.432206,8.00,119.50,152.25,120.75,146.00,146.50,fall
86,151.666667,0.000023,141.9,132.391400,144.705487,190.697903,6.25,83.25,190.75,163.00,143.75,141.25,spring
87,152.333333,0.000009,151.0,90.731776,118.788096,167.554680,10.50,132.25,164.00,132.00,144.75,151.50,fall


In [16]:
cool_only_df

,lips_s,face_l_var,skin_avg_b,R,G,B,H,S,V,L,a,b,label
1,150.000000,0.000017,135.7,146.529861,151.952035,183.768130,50.25,52.50,192.00,174.75,139.25,135.50,winter
3,148.666667,0.000015,131.7,124.505782,121.864497,154.492476,132.00,65.00,159.75,140.25,142.75,131.50,summer
6,152.666667,0.000011,137.9,133.683797,143.018812,176.591909,6.50,75.00,177.00,156.25,139.75,137.75,winter
7,149.000000,0.000013,131.2,153.055289,150.278625,182.753878,177.00,51.75,180.00,161.50,141.75,131.00,summer
9,151.666667,0.000013,141.8,114.381391,130.792897,167.393703,9.75,87.00,162.75,141.00,139.25,141.75,summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,150.666667,0.000011,133.3,140.014710,139.552139,174.484012,90.00,50.00,180.00,163.25,140.25,132.50,winter
175,150.333333,0.000023,136.5,138.447683,143.342302,184.442171,2.75,70.75,180.00,155.00,143.00,136.50,summer
179,151.000000,0.000017,133.4,127.046772,126.131906,161.672470,46.50,71.00,159.25,141.25,140.25,133.00,summer
181,146.000000,0.000021,129.7,151.836415,145.011164,177.402327,174.50,56.75,179.00,161.75,142.00,129.50,summer
